In [1]:
import matplotlib.pyplot as plt
import numpy as np
import polars as pl
import pandas as pd
from tbparse import SummaryReader
from settings import TrainingSettings, DatasetSettings, EvaluationSettings, CNNSettings, CNNMambaSettings, CNNAttentionSettings, CRNNSettings, asdict

In [2]:
def get_columns_with_type(typ) -> list[str]:
    out = []
    classes = [TrainingSettings, DatasetSettings, EvaluationSettings]
    for cls in classes:
        settings = cls()
        dic = asdict(settings)
        for name, value in dic.items():
            if type(value) is typ:
                out.append(name)
    classes = [CNNSettings, CNNAttentionSettings, CNNMambaSettings, CRNNSettings]
    for cls in classes:
        settings = cls(3, 84)
        dic = asdict(settings)
        for name, value in dic.items():
            if type(value) is typ:
                out.append(name)
    return list(set(out))

In [3]:
integer_values = list(set(get_columns_with_type(int) + ["early_stopping"]))
boolean_values = get_columns_with_type(bool)
float_values = get_columns_with_type(float)
string_values = get_columns_with_type(str) + ["dir_name"]
string_values.remove("backbone")

In [4]:
integer_values

['peak_mean_range',
 'd_state',
 'num_heads',
 'context_size',
 'weight_decay',
 'num_conv_layers',
 'fft_size',
 'pr_points',
 'early_stopping',
 'channel_multiplication',
 'num_residual_blocks',
 'classifier_dim',
 'batch_size',
 'test_batch_size',
 'num_attention_blocks',
 'dropout',
 'expansion_factor',
 'd_conv',
 'num_workers',
 'down_sample_factor',
 'num_channels',
 'expand',
 'n_mels',
 'num_feature_layers',
 'sample_rate',
 'peak_max_range',
 'epochs',
 'n_layers',
 'num_rnn_layers',
 'rnn_units',
 'hop_size']

In [5]:
logs = SummaryReader("runs/", pivot=True, extra_columns={'dir_name'})
logs_no_pivot = SummaryReader("runs/", pivot=False, extra_columns={'dir_name'})

In [6]:
# convert types
params = logs.hparams
params = params.loc[:,~params.columns.duplicated()].copy()
params[params == "None"] = None
params[integer_values] = params[integer_values].astype(pd.Int64Dtype(), errors="ignore")
params[boolean_values] = params[boolean_values].astype(bool)
params[float_values] = params[float_values].astype(np.float64)
params[string_values] = params[string_values].astype(pd.StringDtype())
params = pd.DataFrame(params)

In [7]:
params = pl.from_pandas(params, nan_to_null=True)
scores = pl.from_pandas(logs.scalars, nan_to_null=True)
hparams = params.join(scores.select(pl.col("F-Score", "dir_name")), on='dir_name', how='inner')
hparams = hparams.select(pl.all().exclude("dir_name"), pl.col("dir_name").str.split("/").list.first())

# Convert plots
images = logs_no_pivot.images
size = np.array(images.iloc[0]["value"].shape)
images["value"] = images["value"].apply(lambda x: x.flatten())
plots = pl.from_pandas(images, nan_to_null=True)
num_rows = plots.shape[0]
plots = plots.select(pl.col("dir_name"), pl.col("step"), pl.col("tag"),
                     pl.col("value").reshape(tuple([num_rows, *size]), pl.Array).alias("value"))
plots = plots.pivot(values=["value"], columns=["tag"], index=["step", "dir_name"])

ModuleNotFoundError: No module named 'tensorflow'. Please install 'tensorflow' or 'tensorflow-cpu'.

In [8]:
pl.Config.set_tbl_cols(100)
pl.Config.set_tbl_rows(100)


def get_model_settings(model_type: str) -> (pl.DataFrame, pl.DataFrame):
    model = hparams.filter(pl.col("model_settings").str.contains(model_type))
    non_null = model.select(pl.all().is_not_null().all()).row(0)
    model = model[:, non_null]
    different = model.select(pl.all().n_unique() > 1).row(0)
    diff = model[:, different].sort("F-Score", descending=True)
    iden = model.select(pl.all().n_unique() == 1).row(0)
    identical = model[:, iden].select(pl.all().exclude("dir_name", "F-Score")).limit(1)

    return diff, identical


def get_history(name: str) -> pl.DataFrame:
    data = scores.filter(pl.col("dir_name") == name)
    prs = plots.filter(pl.col("dir_name") == name)
    data = data.join(prs, on="step", how="inner")
    return data

In [9]:
unique, identical = get_model_settings("mamba")
print(identical)
unique

shape: (1, 23)
┌─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┐
│ bea ┆ cau ┆ cen ┆ dec ┆ det ┆ ema ┆ fft ┆ ful ┆ hop ┆ ign ┆ mel ┆ mel ┆ min ┆ nor ┆ pad ┆ pad ┆ pad ┆ pea ┆ pea ┆ sam ┆ spl ┆ tra ┆ use │
│ ts  ┆ sal ┆ ter ┆ oup ┆ ect ┆ --- ┆ _si ┆ l_l ┆ _si ┆ ore ┆ _ma ┆ _mi ┆ _sa ┆ mal ┆ _an ┆ _mo ┆ _va ┆ k_m ┆ k_m ┆ ple ┆ its ┆ in_ ┆ _re │
│ --- ┆ --- ┆ --- ┆ led ┆ _to ┆ boo ┆ ze  ┆ eng ┆ ze  ┆ _be ┆ x   ┆ n   ┆ ve_ ┆ ize ┆ not ┆ de  ┆ lue ┆ ax_ ┆ ean ┆ _ra ┆ --- ┆ set ┆ lat │
│ boo ┆ boo ┆ boo ┆ _we ┆ ler ┆ l   ┆ --- ┆ th_ ┆ --- ┆ ats ┆ --- ┆ --- ┆ sco ┆ --- ┆ ati ┆ --- ┆ --- ┆ ran ┆ _ra ┆ te  ┆ str ┆ --- ┆ ive │
│ l   ┆ l   ┆ l   ┆ igh ┆ anc ┆     ┆ i64 ┆ tes ┆ i64 ┆ --- ┆ f64 ┆ f64 ┆ re  ┆ boo ┆ ons ┆ str ┆ f64 ┆ ge  ┆ nge ┆ --- ┆     ┆ str ┆ _po │
│     ┆     ┆     ┆ t_d ┆ e   ┆     ┆     ┆ t   ┆     ┆ boo ┆     ┆     ┆ --- ┆ l   ┆ --- ┆     ┆     ┆ --- ┆ --- ┆ i64 ┆     ┆     ┆ s   │
│    

activation,batch_size,dataset_version,dropout,epochs,flux,learning_rate,mapping,min_test_score,model_settings,n_mels,num_channels,num_workers,onset_cooldown,scheduler,test_batch_size,time_shift,weight_decay,F-Score,dir_name
str,i64,str,f64,i64,bool,f64,str,f64,str,i64,i64,i64,f64,bool,i64,f64,f64,f64,str
"""SELU""",40,"""M""",0.1,50,true,0.00006,"""Three class standard""",0.48,"""mamba_fast""",84,16,16,0.021,false,3,0.015,0.0001,0.508625,"""Jul31_09-51-49_marclie-desktop"""
"""ELU""",256,"""M""",0.39906,20,false,0.000113,"""Three class standard""",0.48,"""mamba_fast""",128,45,64,0.021,false,1,0.015,0.000452,0.506204,"""Aug05_12-28-01_seppel-liemarce"""
"""SELU""",16,"""M""",0.1,20,true,0.0001,"""Three class standard""",0.48,"""mamba_fast""",84,48,64,0.021,true,1,0.015,0.00001,0.503114,"""Jul02_17-30-41_seppel-liemarce"""
"""ELU""",512,"""M""",0.121478,20,true,0.000637,"""Three class standard""",0.48,"""mamba_fast""",128,42,64,0.021,false,1,0.015,0.001411,0.501684,"""Aug10_10-13-52_seppel-liemarce"""
"""SELU""",40,"""M""",0.1,20,false,0.00006,"""Three class standard""",0.48,"""mamba_fast""",84,16,16,0.021,false,3,0.015,0.0001,0.50072,"""Jul30_21-32-14_marclie-desktop"""
"""SELU""",64,"""L""",0.1,30,false,0.0001,"""Three class standard""",0.48,"""mamba_fast""",84,32,16,0.021,true,1,0.015,0.00001,0.500146,"""Jun16_11-35-01_marclie-desktop"""
"""ELU""",256,"""M""",0.139294,20,true,0.000671,"""Three class standard""",0.48,"""mamba_fast""",128,63,64,0.021,false,1,0.015,0.000243,0.499378,"""Aug09_13-39-49_seppel-liemarce"""
"""ELU""",512,"""M""",0.08146,20,true,0.001594,"""Three class standard""",0.48,"""mamba_fast""",128,50,64,0.021,false,1,0.015,0.001379,0.499168,"""Aug09_18-59-58_seppel-liemarce"""
"""SELU""",512,"""M""",0.162201,20,true,0.001021,"""Three class standard""",0.48,"""mamba_fast""",96,31,64,0.021,false,1,0.015,0.000014,0.496763,"""Aug10_19-41-07_seppel-liemarce"""


In [11]:
unique, identical = get_model_settings("attention")
print(identical)
unique

shape: (1, 21)
┌─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┐
│ act ┆ bea ┆ cen ┆ det ┆ dro ┆ ema ┆ fft ┆ hop ┆ ign ┆ mel ┆ mel ┆ min ┆ mod ┆ nor ┆ pad ┆ pad ┆ pea ┆ pea ┆ sam ┆ spl ┆ tra │
│ iva ┆ ts  ┆ ter ┆ ect ┆ pou ┆ --- ┆ _si ┆ _si ┆ ore ┆ _ma ┆ _mi ┆ _sa ┆ el_ ┆ mal ┆ _mo ┆ _va ┆ k_m ┆ k_m ┆ ple ┆ its ┆ in_ │
│ tio ┆ --- ┆ --- ┆ _to ┆ t   ┆ boo ┆ ze  ┆ ze  ┆ _be ┆ x   ┆ n   ┆ ve_ ┆ set ┆ ize ┆ de  ┆ lue ┆ ax_ ┆ ean ┆ _ra ┆ --- ┆ set │
│ n   ┆ boo ┆ boo ┆ ler ┆ --- ┆ l   ┆ --- ┆ --- ┆ ats ┆ --- ┆ --- ┆ sco ┆ tin ┆ --- ┆ --- ┆ --- ┆ ran ┆ _ra ┆ te  ┆ str ┆ --- │
│ --- ┆ l   ┆ l   ┆ anc ┆ f64 ┆     ┆ i64 ┆ i64 ┆ --- ┆ f64 ┆ f64 ┆ re  ┆ gs  ┆ boo ┆ str ┆ f64 ┆ ge  ┆ nge ┆ --- ┆     ┆ str │
│ str ┆     ┆     ┆ e   ┆     ┆     ┆     ┆     ┆ boo ┆     ┆     ┆ --- ┆ --- ┆ l   ┆     ┆     ┆ --- ┆ --- ┆ i64 ┆     ┆     │
│     ┆     ┆     ┆ --- ┆     ┆     ┆     ┆     ┆ l   ┆     ┆     ┆ f64 ┆ str ┆     ┆    

batch_size,causal,context_size,dataset_version,decoupled_weight_decay,epochs,flux,full_length_test,learning_rate,mapping,min_test_score,n_mels,num_attention_blocks,num_channels,num_heads,num_workers,onset_cooldown,pad_annotations,scheduler,test_batch_size,time_shift,use_relative_pos,F-Score,dir_name
i64,bool,i64,str,bool,i64,bool,bool,f64,str,f64,i64,i64,i64,i64,i64,f64,bool,bool,i64,f64,bool,f64,str
16,true,200,"""M""",true,20,true,true,0.0001,"""Three class standard""",0.54,84,5,24,8,16,0.021,true,true,1,0.015,false,0.493905,"""Jun15_14-16-35_marclie-desktop"""
16,true,200,"""L""",true,20,true,true,0.0001,"""THREE_CLASS_STANDARD""",0.54,84,5,24,8,64,0.021,true,true,1,0.015,false,0.492189,"""Jun13_10-57-29_seppel-liemarce"""
16,true,200,"""L""",true,20,true,true,0.0001,"""THREE_CLASS_STANDARD""",0.54,84,5,32,8,64,0.021,true,true,1,0.015,false,0.482672,"""Jun14_09-33-24_seppel-liemarce"""
4,true,200,"""L""",true,20,false,true,0.0001,"""THREE_CLASS_STANDARD""",0.54,84,5,16,8,64,0.021,true,true,1,0.015,false,0.458652,"""Jun11_13-54-21_seppel-liemarce"""
16,true,200,"""M""",true,20,true,true,0.0001,"""Three class standard""",0.48,84,4,24,8,16,0.021,true,true,1,0.015,false,0.452022,"""Jun15_18-05-40_marclie-desktop"""
16,true,200,"""L""",true,20,true,true,0.0001,"""Three class standard""",0.48,84,5,24,8,64,0.021,true,true,1,0.02,false,0.393078,"""Jun17_15-04-11_seppel-liemarce"""
4,true,200,"""S""",true,20,false,true,0.0001,"""THREE_CLASS_STANDARD""",0.54,84,5,32,8,64,0.021,true,true,1,0.015,false,0.248427,"""Jun11_06-49-13_seppel-liemarce"""
16,true,200,"""M""",true,20,true,true,0.0001,"""Three class standard""",0.48,84,5,24,8,16,0.021,true,true,1,0.015,false,0.248362,"""Jun15_20-26-24_marclie-desktop"""
8,true,200,"""L""",true,20,true,true,0.0001,"""THREE_CLASS_STANDARD""",0.54,84,5,24,8,64,0.021,true,true,1,0.015,false,0.233853,"""Jun12_07-38-49_seppel-liemarce"""


In [10]:
unique, identical = get_model_settings("cnn")
print(identical)
unique

shape: (1, 14)
┌─────┬─────┬─────┬─────┬─────┬─────┬─────┬────────┬───────┬───────┬───────┬───────┬───────┬───────┐
│ bea ┆ cen ┆ fft ┆ hop ┆ ign ┆ mel ┆ mel ┆ normal ┆ pad_m ┆ peak_ ┆ peak_ ┆ sampl ┆ split ┆ train │
│ ts  ┆ ter ┆ _si ┆ _si ┆ ore ┆ _ma ┆ _mi ┆ ize    ┆ ode   ┆ max_r ┆ mean_ ┆ e_rat ┆ s     ┆ _set  │
│ --- ┆ --- ┆ ze  ┆ ze  ┆ _be ┆ x   ┆ n   ┆ ---    ┆ ---   ┆ ange  ┆ range ┆ e     ┆ ---   ┆ ---   │
│ boo ┆ boo ┆ --- ┆ --- ┆ ats ┆ --- ┆ --- ┆ bool   ┆ str   ┆ ---   ┆ ---   ┆ ---   ┆ str   ┆ str   │
│ l   ┆ l   ┆ i64 ┆ i64 ┆ --- ┆ f64 ┆ f64 ┆        ┆       ┆ i64   ┆ i64   ┆ i64   ┆       ┆       │
│     ┆     ┆     ┆     ┆ boo ┆     ┆     ┆        ┆       ┆       ┆       ┆       ┆       ┆       │
│     ┆     ┆     ┆     ┆ l   ┆     ┆     ┆        ┆       ┆       ┆       ┆       ┆       ┆       │
╞═════╪═════╪═════╪═════╪═════╪═════╪═════╪════════╪═══════╪═══════╪═══════╪═══════╪═══════╪═══════╡
│ fal ┆ tru ┆ 204 ┆ 441 ┆ tru ┆ 200 ┆ 20. ┆ false  ┆ const ┆ 2     ┆ 2     ┆

activation,batch_size,causal,dataset_version,decoupled_weight_decay,detect_tolerance,dropout,ema,epochs,flux,full_length_test,learning_rate,mapping,min_save_score,min_test_score,model_settings,n_mels,num_channels,num_workers,onset_cooldown,pad_annotations,pad_value,scheduler,test_batch_size,time_shift,use_relative_pos,F-Score,dir_name
str,i64,bool,str,bool,f64,f64,bool,i64,bool,bool,f64,str,f64,f64,str,i64,i64,i64,f64,bool,f64,bool,i64,f64,bool,f64,str
"""SELU""",1024,false,"""M""",true,0.05,0.037324,false,20,true,true,0.002284,"""Three class standard""",0.62,0.58,"""cnn""",128,18,64,0.021,true,0.5,false,1,0.0,true,0.614844,"""Aug18_20-04-32_seppel-liemarce"""
"""SELU""",1024,false,"""M""",true,0.025,0.11919,false,20,true,true,0.000888,"""Three class standard""",0.62,0.58,"""cnn""",128,18,64,0.021,true,0.5,false,1,0.0,true,0.61134,"""Aug18_11-35-22_seppel-liemarce"""
"""SELU""",512,false,"""M""",true,0.05,0.094742,false,20,true,true,0.000654,"""Three class standard""",0.62,0.58,"""cnn""",84,30,64,0.021,true,0.5,false,1,0.0,true,0.608412,"""Aug20_03-55-01_seppel-liemarce"""
"""SELU""",512,false,"""M""",true,0.05,0.092661,false,20,true,true,0.001075,"""Three class standard""",0.62,0.58,"""cnn""",84,32,64,0.021,true,0.5,false,1,0.0,true,0.607103,"""Aug20_02-04-31_seppel-liemarce"""
"""SELU""",512,false,"""M""",true,0.05,0.139728,false,20,true,true,0.000746,"""Three class standard""",0.62,0.58,"""cnn""",84,35,64,0.021,true,0.5,false,1,0.0,true,0.606601,"""Aug20_09-04-48_seppel-liemarce"""
"""SELU""",1024,false,"""M""",true,0.025,0.121737,false,20,true,true,0.002615,"""Three class standard""",0.62,0.58,"""cnn""",128,19,64,0.021,true,0.5,false,1,0.0,true,0.605724,"""Aug18_13-22-48_seppel-liemarce"""
"""SELU""",512,false,"""M""",true,0.05,0.097741,false,20,true,true,0.000408,"""Three class standard""",0.62,0.58,"""cnn""",84,32,64,0.021,true,0.5,false,1,0.0,true,0.601382,"""Aug20_14-20-34_seppel-liemarce"""
"""SELU""",256,false,"""M""",true,0.025,0.34,false,20,false,true,0.0002,"""Three class standard""",0.62,0.48,"""cnn""",128,25,64,0.021,true,0.5,false,1,0.0,true,0.600748,"""Aug14_16-53-12_seppel-liemarce"""
"""SELU""",512,false,"""M""",true,0.05,0.037476,false,20,true,true,0.001877,"""Three class standard""",0.62,0.58,"""cnn""",84,29,64,0.021,true,0.5,false,1,0.0,true,0.600425,"""Aug20_00-09-38_seppel-liemarce"""


In [24]:
unique, identical = get_model_settings("crnn")
print(identical)
unique

shape: (1, 33)
┌─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┐
│ bea ┆ cau ┆ cen ┆ det ┆ ema ┆ fft ┆ ful ┆ hop ┆ ign ┆ lab ┆ lab ┆ map ┆ mel ┆ mel ┆ min ┆ min ┆ mod ┆ nor ┆ ons ┆ pad ┆ pad ┆ pad ┆ pea ┆ pea ┆ pos ┆ sam ┆ sch ┆ seg ┆ spl ┆ tes ┆ tim ┆ tra ┆ use │
│ ts  ┆ sal ┆ ter ┆ ect ┆ --- ┆ _si ┆ l_l ┆ _si ┆ ore ┆ el_ ┆ el_ ┆ pin ┆ _ma ┆ _mi ┆ _sa ┆ _te ┆ el_ ┆ mal ┆ et_ ┆ _an ┆ _mo ┆ _va ┆ k_m ┆ k_m ┆ iti ┆ ple ┆ edu ┆ men ┆ its ┆ t_b ┆ e_s ┆ in_ ┆ _re │
│ --- ┆ --- ┆ --- ┆ _to ┆ boo ┆ ze  ┆ eng ┆ ze  ┆ _be ┆ lea ┆ lea ┆ g   ┆ x   ┆ n   ┆ ve_ ┆ st_ ┆ set ┆ ize ┆ coo ┆ not ┆ de  ┆ lue ┆ ax_ ┆ ean ┆ ve_ ┆ _ra ┆ ler ┆ t_t ┆ --- ┆ atc ┆ hif ┆ set ┆ lat │
│ boo ┆ boo ┆ boo ┆ ler ┆ l   ┆ --- ┆ th_ ┆ --- ┆ ats ┆ d_i ┆ d_o ┆ --- ┆ --- ┆ --- ┆ sco ┆ sco ┆ tin ┆ --- ┆ ldo ┆ ati ┆ --- ┆ --- ┆ ran ┆ _ra ┆ wei ┆ te  ┆ --- ┆ ype ┆ str ┆ h_s ┆ t  

activation,batch_size,channel_multiplication,classifier_dim,dataset_version,decoupled_weight_decay,down_sample_factor,dropout,epochs,flux,frame_length,frame_overlap,learning_rate,n_mels,num_channels,num_conv_layers,num_rnn_layers,num_workers,pr_points,rnn_units,weight_decay,F-Score,dir_name
str,i64,i64,i64,str,bool,i64,f64,i64,bool,f64,f64,f64,i64,i64,i64,i64,i64,i64,i64,f64,f64,str
"""ELU""",9,4,408,"""S""",true,3,0.0,20,false,2.0,0.2,0.00012,128,32,2,5,16,1000,1337,0.0,0.617189,"""Oct06_11-36-53_marclie-desktop"""
"""ELU""",36,1,298,"""M""",true,3,0.2,30,false,1.2,0.4,0.00012,96,24,1,5,64,400,1740,0.000157,0.614033,"""Sep07_00-46-03_seppel-liemarce"""
"""ELU""",9,3,111,"""M""",true,3,0.0,30,false,2.1,0.2,0.000123,128,29,3,5,64,400,1559,1.5000e-10,0.612625,"""Sep20_12-33-51_seppel-liemarce"""
"""ELU""",9,4,408,"""M""",true,3,0.0,30,false,1.9,0.2,0.000128,128,29,2,5,64,400,1337,4.7523e-10,0.610186,"""Sep19_04-27-08_seppel-liemarce"""
"""SELU""",11,4,30,"""M""",false,2,0.05,30,false,1.5,0.3,0.000063,64,24,4,6,64,400,2038,0.000002,0.606079,"""Sep12_11-56-45_seppel-liemarce"""
"""SELU""",10,4,365,"""M""",true,3,0.0,30,false,1.7,0.6,0.000087,64,27,2,4,64,400,1209,0.000051,0.603451,"""Sep04_13-45-32_seppel-liemarce"""
"""ReLU""",11,1,282,"""M""",true,2,0.0,30,false,2.7,0.3,0.000247,128,29,1,5,64,400,1056,3.3577e-8,0.600424,"""Aug28_12-46-41_seppel-liemarce"""
"""ELU""",10,1,488,"""M""",true,3,0.0,30,false,1.6,0.3,0.000115,128,32,1,5,64,400,923,2.8259e-10,0.600182,"""Oct01_08-56-15_seppel-liemarce"""
"""SELU""",23,4,666,"""M""",false,4,0.0,30,false,1.8,0.5,0.000091,64,30,2,4,64,400,856,0.00004,0.599414,"""Sep15_18-38-10_seppel-liemarce"""


In [25]:
best = unique.select("dir_name").row(0)[0]
history = get_history(best)
pr_curve = history.select(pl.col("Validation/PR-Curve/")).row(-1)[0]

NameError: name 'plots' is not defined

In [ ]:
plt.imshow(pr_curve)
plt.axis("off")

In [17]:
def get_model_dict(unique: pl.DataFrame, identical: pl.DataFrame) -> dict:
    dics = []
    for row in unique.to_dicts():
        dics.append(row | identical.to_dicts()[0])
    return dics

In [21]:
get_model_dict(unique, identical)[0]

{'activation': 'ELU',
 'batch_size': 9,
 'channel_multiplication': 4,
 'classifier_dim': 408,
 'dataset_version': 'S',
 'decoupled_weight_decay': True,
 'down_sample_factor': 3,
 'dropout': 0.0,
 'epochs': 20,
 'flux': False,
 'frame_length': 2.0,
 'frame_overlap': 0.2,
 'learning_rate': 0.00012,
 'n_mels': 128,
 'num_channels': 32,
 'num_conv_layers': 2,
 'num_rnn_layers': 5,
 'num_workers': 16,
 'pr_points': 1000,
 'rnn_units': 1337,
 'weight_decay': 0.0,
 'F-Score': 0.617189347743988,
 'dir_name': 'Oct06_11-36-53_marclie-desktop',
 'beats': False,
 'causal': True,
 'center': True,
 'detect_tolerance': 0.025,
 'ema': False,
 'fft_size': 2048,
 'full_length_test': True,
 'hop_size': 441,
 'ignore_beats': True,
 'label_lead_in': 0.25,
 'label_lead_out': 0.1,
 'mapping': 'Three class standard',
 'mel_max': 20000.0,
 'mel_min': 20.0,
 'min_save_score': 0.62,
 'min_test_score': 0.48,
 'model_settings': 'crnn',
 'normalize': False,
 'onset_cooldown': 0.021,
 'pad_annotations': True,
 'pad_

In [43]:
# Get Value
params.loc[params["dir_name"].str.contains(best)][["full_length_test"]]

,full_length_test
359,True
